In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

In [3]:
ds= pd.read_csv('/content/drive/MyDrive/diabetes_data.csv')
ds.head()

,Age,Gender,Polyuria,Polydipsia,Sudden_weight_loss,weakness,Polyphagia,Genital_thrush,visual_blurring,Itching,Irritability,delayed_healing,partial_paresis,muscle_stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In [4]:
ds.dtypes

Age                    int64
Gender                object
Polyuria              object
Polydipsia            object
Sudden_weight_loss    object
weakness              object
Polyphagia            object
Genital_thrush        object
visual_blurring       object
Itching               object
Irritability          object
delayed_healing       object
partial_paresis       object
muscle_stiffness      object
Alopecia              object
Obesity               object
class                 object
dtype: object

In [5]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Age                 520 non-null    int64 
 1   Gender              520 non-null    object
 2   Polyuria            520 non-null    object
 3   Polydipsia          520 non-null    object
 4   Sudden_weight_loss  520 non-null    object
 5   weakness            520 non-null    object
 6   Polyphagia          520 non-null    object
 7   Genital_thrush      520 non-null    object
 8   visual_blurring     520 non-null    object
 9   Itching             520 non-null    object
 10  Irritability        520 non-null    object
 11  delayed_healing     520 non-null    object
 12  partial_paresis     520 non-null    object
 13  muscle_stiffness    520 non-null    object
 14  Alopecia            520 non-null    object
 15  Obesity             520 non-null    object
 16  class               520 no

In [6]:
import re
def func(x):
  if x == "Male":
    return "1"
  else:
    return "0" 

ds['Gender']=ds['Gender'].apply(lambda x : func(x))



In [7]:
import re
def func(x):
  if x == "Positive":
    return "1"
  else:
    return "0" 

ds['class']=ds['class'].apply(lambda x : func(x))

In [8]:
import re
def func(x):
  if x == "Yes":
    return "1"
  else:
    return "0" 
ds['Polyuria']=ds['Polyuria'].apply(lambda x : func(x))
ds['Polydipsia']=ds['Polydipsia'].apply(lambda x : func(x))
ds['Sudden_weight_loss']=ds['Sudden_weight_loss'].apply(lambda x : func(x))
ds['weakness']=ds['weakness'].apply(lambda x : func(x))
ds['Polyphagia']=ds['Polyphagia'].apply(lambda x : func(x))
ds['Genital_thrush']=ds['Genital_thrush'].apply(lambda x : func(x))
ds['visual_blurring']=ds['visual_blurring'].apply(lambda x : func(x))
ds['Itching']=ds['Itching'].apply(lambda x : func(x))
ds['Irritability']=ds['Irritability'].apply(lambda x : func(x))
ds['delayed_healing']=ds['delayed_healing'].apply(lambda x : func(x)) 
ds['partial_paresis']=ds['partial_paresis'].apply(lambda x : func(x))
ds['muscle_stiffness']=ds['muscle_stiffness'].apply(lambda x : func(x))  
ds['Alopecia']=ds['Alopecia'].apply(lambda x : func(x))
ds['Obesity']=ds['Obesity'].apply(lambda x : func(x))

In [9]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
for column in ds.columns:
    ds[column] = labelencoder.fit_transform(ds[column])

In [10]:
factor_columns=ds[['Age', 'Gender', 'Polyuria', 'Polydipsia', 'Sudden_weight_loss',
       'weakness', 'Polyphagia', 'Genital_thrush', 'visual_blurring',
       'Itching', 'Irritability', 'delayed_healing', 'partial_paresis',
       'muscle_stiffness', 'Alopecia', 'Obesity']]

In [11]:
dia_result=ds['class']
dia_result.head()

0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split( factor_columns, dia_result, test_size=0.3, random_state=5) 
X_train, X_test_validation, y_train, y_test_validation = train_test_split(X_train, y_train, test_size=0.3)

In [16]:
batch_size=64
epochs=50

max_features = 5000
maxlen= 1045
#batch_size=256
embedding_dims=16
filters=128
kernel_size=3
hidden_size=128


In [17]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv1D,Flatten, MaxPooling1D, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 

In [18]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [19]:
model = Sequential()

model.add(Conv1D(filters = 256, kernel_size=3, activation='relu', input_shape=(16,1)))
  #Adding MaxPool layer with pool_size as 2
model.add(MaxPooling1D(pool_size =2))
  #Adding 2nd convolution layer
  

model.add(Conv1D(filters = 128, kernel_size=5, activation='relu'))
  #Adding MaxPool layer with pool_size as 2
model.add(MaxPooling1D(pool_size =2))
  
model.add(Flatten())
  #Adding Fully connected layer

model.add(Dense(512, activation='relu'))

model.add(Dense(256, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(64, activation='relu'))


#Adding Dropout layer
model.add(Dropout(0.7))
  

model.add(Dense(2, activation='softmax'))



In [20]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy',f1_m,precision_m,recall_m])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 14, 256)           1024      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 7, 256)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 3, 128)            163968    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 1, 128)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 512)               6

In [21]:
model.fit(X_train,y_train,epochs=epochs,validation_data=(X_test_validation, y_test_validation),batch_size=batch_size,verbose=2)

Epoch 1/50
4/4 - 3s - loss: 0.6944 - accuracy: 0.5669 - f1_m: 0.7605 - precision_m: 0.6144 - recall_m: 1.0000 - val_loss: 0.6245 - val_accuracy: 0.6636 - val_f1_m: 0.8036 - val_precision_m: 0.6743 - val_recall_m: 1.0000 - 3s/epoch - 690ms/step
Epoch 2/50
4/4 - 0s - loss: 0.6424 - accuracy: 0.6142 - f1_m: 0.7601 - precision_m: 0.6139 - recall_m: 1.0000 - val_loss: 0.5797 - val_accuracy: 0.6636 - val_f1_m: 0.8036 - val_precision_m: 0.6743 - val_recall_m: 1.0000 - 324ms/epoch - 81ms/step
Epoch 3/50
4/4 - 0s - loss: 0.5952 - accuracy: 0.6575 - f1_m: 0.7609 - precision_m: 0.6143 - recall_m: 1.0000 - val_loss: 0.5308 - val_accuracy: 0.8182 - val_f1_m: 0.8036 - val_precision_m: 0.6743 - val_recall_m: 1.0000 - 309ms/epoch - 77ms/step
Epoch 4/50
4/4 - 0s - loss: 0.5495 - accuracy: 0.7283 - f1_m: 0.7600 - precision_m: 0.6138 - recall_m: 1.0000 - val_loss: 0.4533 - val_accuracy: 0.8455 - val_f1_m: 0.8036 - val_precision_m: 0.6743 - val_recall_m: 1.0000 - 323ms/epoch - 81ms/step
Epoch 5/50
4/4 - 0

In [22]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy " + str(accuracy) + ":\n\ff1_score = " + str(f1_score) +
          "\nPrecision = " + str(precision) + "\nRecall = " + str(recall))

Accuracy 0.9102563858032227:
f1_score = 0.733849287033081
Precision = 0.5821428298950195
Recall = 1.0


In [23]:
prediction_features = model.predict(X_test)
performance = model.evaluate(X_test, y_test)
print(performance)

5/5 [==============================] - 0s 4ms/step - loss: 0.5113 - accuracy: 0.9103 - f1_m: 0.7338 - precision_m: 0.5821 - recall_m: 1.0000
[0.5112795233726501, 0.9102563858032227, 0.733849287033081, 0.5821428298950195, 1.0]
